### Import Libraries ###

In [96]:
import pydicom
import numpy as np
import cv2
from glob import glob
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

import os, glob, json, random, math
import numpy as np
import pydicom, cv2
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.cuda.amp import autocast, GradScaler


### Load Image Dataset ###

In [64]:
# reads a single dicom image and converts it to a numpy array
def load_dicom_image(file_path):
    """Load a DICOM image and return it as a numpy array."""
    dicom = pydicom.dcmread(file_path)
    image = dicom.pixel_array
    return image

def preprocess_image(image, target_size=(256, 256)):
    """Preprocess the image: resize and normalize."""
    image_resized = cv2.resize(image, target_size)
    image_normalized = (image_resized - np.min(image_resized)) / (np.max(image_resized) - np.min(image_resized))
    return image_normalized

# reads all dicom images from a directory and preprocesses them
def load_dicom_images_from_directory(directory):
    """Load and preprocess all DICOM images from a directory."""
    file_paths = glob(os.path.join(directory, '*.dcm'))
    images = []
    for file_path in file_paths:
        image = load_dicom_image(file_path)
        image_preprocessed = preprocess_image(image)
        images.append(image_preprocessed)
    return np.array(images)

In [65]:
# testing the functions (IGNORE THIS)

dicom_files = glob(os.path.join('path_to_dicom_files', '*.dcm'))
for file in dicom_files:
    img = load_dicom_image(file)
    img_preprocessed = preprocess_image(img)
    print(f"Processed image shape: {img_preprocessed.shape}")
    break  # Just process one file for demonstration

# --- PATH TO YOUR DATA ---
root_folder = "/Users/jinkyungjeon/cs3244_project/osic-pulmonary-fibrosis-progression/train"

# Choose one patient
patient_id = "ID00184637202242062969203"  # change this to any patient folder name
patient_folder = os.path.join(root_folder, patient_id)

# Get all DICOM file paths in that patient's folder
dicom_files = sorted([os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith(".dcm")])

print(f"Patient: {patient_id}")
print(f"Found {len(dicom_files)} DICOM slices")

Patient: ID00184637202242062969203
Found 62 DICOM slices


### Preprocessing

In [66]:
import os, glob, pydicom, numpy as np, cv2, random, json
from typing import List, Dict
from torch.utils.data import Dataset, DataLoader
import torch

def dicom_to_hu(path: str) -> np.ndarray:
    """Convert DICOM pixel data to Hounsfield Units (HU)."""
    dicom = pydicom.dcmread(path)
    image = dicom.pixel_array.astype(np.int16)

    # Convert to HU
    intercept = dicom.RescaleIntercept
    slope = dicom.RescaleSlope 

    image = slope * image + intercept

    return image

def lung_window(img_hu: np.ndarray, center = -600, width = 1500) -> np.ndarray:
    """Apply lung windowing to the HU image."""

    min_hu = center - (width // 2)
    max_hu = center + (width // 2)

    img_windowed = np.clip(img_hu, min_hu, max_hu) 

    return img_windowed

def load_patient_stack(dicom_dir: str, target_depth: int = 96, img_size: int = 224) -> np.ndarray:
    """Load all slices, sort by InstanceNumber, window, resize, depth-pad/trim."""
    files = glob.glob(os.path.join(dicom_dir, "*.dcm"))
    if not files:
        raise FileNotFoundError(f"No DICOMs in {dicom_dir}")

    # sort slices using InstanceNumber or filename fallback
    def inst_no(fp):
        try:
            return int(pydicom.dcmread(fp, stop_before_pixels=True).InstanceNumber)
        except Exception:
            return 0
    files.sort(key=inst_no)

    slices = [lung_window(dicom_to_hu(fp)) for fp in files] # apply lung windowing to each slice converted to HU

    # resize each slice to cnn standard image size
    slices = [cv2.resize(s, (img_size, img_size), interpolation=cv2.INTER_AREA) for s in slices]
    vol = np.stack(slices, axis=0)  # [D, H, W]

    # normalize per-volume (optional but helpful)
    v = vol.astype(np.float32)
    v = (v - v.mean()) / (v.std() + 1e-6)

    # depth pad/trim to target_depth
    D = v.shape[0]
    if D == target_depth:
        pass
    elif D > target_depth:
        # uniform downsample to target_depth
        idx = np.linspace(0, D-1, target_depth).round().astype(int)
        v = v[idx]
    else:
        # pad by symmetric reflection
        pad_needed = target_depth - D
        pre = pad_needed // 2
        post = pad_needed - pre
        v = np.pad(v, ((pre, post), (0,0), (0,0)), mode="reflect")

    # channel-first for 2D CNN over slices: keep as [D, H, W]; we’ll add channel later
    return v  # float32

class OSICMini(Dataset):
    def __init__(self, rows: List[Dict], depth=96, size=224, aug=False):
        self.rows = rows; self.depth = depth; self.size = size; self.aug = aug

    def __len__(self): return len(self.rows)

    def __getitem__(self, i):
        r = self.rows[i]
        vol = load_stack(r["dicom_dir"], self.depth, self.size)  # [D,H,W]
        # simple flips
        if self.aug:
            if random.random() < 0.5: vol = np.flip(vol, 1).copy()
            if random.random() < 0.5: vol = np.flip(vol, 2).copy()
        x = torch.from_numpy(vol)[None, ...]  # [1,D,H,W]
        y = torch.tensor(float(r["fvc"]), dtype=torch.float32)
        return x, y

def make_loader(rows, batch_size=2, shuffle=True, num_workers=4, **kw):
    ds = OSICMini(rows, **kw)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                      num_workers=num_workers, pin_memory=True)


### Model: Basic CNN architecture (slice encoder) ###

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicSliceCNN(nn.Module):
    def __init__(self, in_ch=1, out_dim=256):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, 32, 3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(256, out_dim)

    def forward(self, x):  # [B*D,1,H,W]
        f = self.conv(x).flatten(1)
        return self.fc(f)

class DepthAggregator(nn.Module):
    def __init__(self, in_dim=256, hidden=256):
        super().__init__()
        self.conv = nn.Conv1d(in_dim, hidden, kernel_size=5, padding=2)
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc  = nn.Linear(hidden, hidden)

    def forward(self, feat_seq):
        x = feat_seq.transpose(1, 2)
        x = F.relu(self.conv(x))
        x = self.gap(x).squeeze(-1)
        return F.relu(self.fc(x))

class GaussianHead(nn.Module):
    def __init__(self, in_dim=256):
        super().__init__()
        self.mu = nn.Linear(in_dim, 1)
        self.log_var = nn.Linear(in_dim, 1)
    def forward(self, x):
        mu = self.mu(x).squeeze(1)
        log_var = self.log_var(x).squeeze(1)
        log_var = torch.clamp(log_var, -5.0, 5.0)
        return mu, log_var

class OSICProbCNN(nn.Module):
    def __init__(self, slice_dim=256, agg_dim=256):
        super().__init__()
        self.encoder = BasicSliceCNN(in_ch=1, out_dim=slice_dim)
        self.agg = DepthAggregator(in_dim=slice_dim, hidden=agg_dim)
        self.head = GaussianHead(in_dim=agg_dim)
    
    def forward(self, x):
        B,C,D,H,W = x.shape
        x = x.permute(0,2,1,3,4).contiguous()
        x = x.view(B*D,1,H,W)
        feats = self.encoder(x)
        feats = feats.view(B,D,-1)
        g = self.agg(feats)
        mu, log_var = self.head(g)
        return mu, log_var

### LOSS

In [78]:
import math
import torch
import torch.nn as nn

class GaussianNLL(nn.Module):
    def forward(self, mu, log_var, y):
        inv_var = torch.exp(-log_var)
        nll = 0.5 * ((y - mu)**2 * inv_var + log_var + math.log(2*math.pi))
        return nll.mean()

@torch.no_grad()
def gaussian_crps(mu, sigma, y):
    from torch.distributions.normal import Normal
    eps = 1e-7
    sigma = torch.clamp(sigma, min=eps)
    z = (y - mu) / sigma
    normal0 = Normal(torch.zeros_like(z), torch.ones_like(z))
    pdf = torch.exp(normal0.log_prob(z))
    cdf = normal0.cdf(z)
    return (sigma * (z*(2*cdf - 1) + 2*pdf - 1/math.sqrt(math.pi))).mean()

@torch.no_grad()
def gaussian_quantiles(mu, sigma, qs=(0.05,0.5,0.95)):
    z = {0.05:-1.6449, 0.5:0.0, 0.95:1.6449}
    return [mu + sigma*z[q] for q in qs]

### Dataset Loader ###

In [79]:
def _read_hu(fp):
    d = pydicom.dcmread(fp)
    arr = d.pixel_array.astype(np.int16)
    slope = float(getattr(d, "RescaleSlope", 1.0))
    intercept = float(getattr(d, "RescaleIntercept", 0.0))
    return arr * slope + intercept

def _window_norm(img_hu, center=-600, width=1500):
    low, high = center - width/2, center + width/2
    img = np.clip(img_hu, low, high)
    return (img - low) / (high - low + 1e-6)

def _slice_key(fp):
    try: return int(pydicom.dcmread(fp, stop_before_pixels=True).InstanceNumber)
    except: return 0

def load_stack(dicom_dir, target_depth=96, img_size=224):
    fps = sorted(glob.glob(os.path.join(dicom_dir, "*.dcm")), key=_slice_key)
    slices = [_window_norm(_read_hu(fp)) for fp in fps]
    slices = [cv2.resize(s, (img_size, img_size)) for s in slices]
    vol = np.stack(slices, axis=0).astype(np.float32)
    vol = (vol - vol.mean()) / (vol.std() + 1e-6)
    D = vol.shape[0]
    if D > target_depth:
        idx = np.linspace(0, D-1, target_depth).round().astype(int)
        vol = vol[idx]
    elif D < target_depth:
        pad = target_depth - D
        pre = pad // 2; post = pad - pre
        vol = np.pad(vol, ((pre, post), (0,0), (0,0)), mode="reflect")
    return vol

class OSICDataset(Dataset):
    def __init__(self, rows, depth=96, size=224, aug=False):
        self.rows = rows; self.depth = depth; self.size = size; self.aug = aug
    def __len__(self): return len(self.rows)
    def __getitem__(self, i):
        r = self.rows[i]
        vol = load_stack(r["dicom_dir"], self.depth, self.size)
        if self.aug:
            if random.random() < 0.5: vol = np.flip(vol, 1).copy()
            if random.random() < 0.5: vol = np.flip(vol, 2).copy()
        x = torch.from_numpy(vol)[None, ...]  # [1,D,H,W]
        y = torch.tensor(float(r["fvc"]), dtype=torch.float32)
        return x, y

def make_loader(rows, batch_size=2, shuffle=True, num_workers=2, **kw):
    ds = OSICDataset(rows, **kw)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

### Train / Evaluate

In [80]:
def train_one_epoch(model, loader, opt, loss_fn, device, scaler=None, clip=1.0):
    model.train()
    total = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        opt.zero_grad(set_to_none=True)
        if scaler:
            with autocast():
                mu, log_var = model(x)
                loss = loss_fn(mu, log_var, y)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            scaler.step(opt); scaler.update()
        else:
            mu, log_var = model(x)
            loss = loss_fn(mu, log_var, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()
        total += loss.item() * x.size(0)
    return total / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    nlls, crpss = [], []
    cover90, total = 0, 0
    nll_fn = GaussianNLL()
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        mu, log_var = model(x)
        nll = nll_fn(mu, log_var, y).item()
        sigma = torch.sqrt(torch.exp(log_var))
        crps = gaussian_crps(mu, sigma, y).item()
        q05, q50, q95 = gaussian_quantiles(mu, sigma)
        inside = ((y >= q05) & (y <= q95)).float().sum().item()
        cover90 += inside; total += y.numel()
        nlls.append(nll); crpss.append(crps)
    return {"NLL": np.mean(nlls), "CRPS": np.mean(crpss), "PI90": cover90/total}


In [81]:
import os, glob, pandas as pd, numpy as np, torch, cv2, pydicom, random
from torch.utils.data import Dataset, DataLoader

# paths
DATA_DIR = "/Users/jinkyungjeon/cs3244_project/osic-pulmonary-fibrosis-progression" # root folder containing train/ and test/
CSV_PATH = os.path.join(DATA_DIR, "train.csv")

# read CSV
df = pd.read_csv(CSV_PATH)
print(df.head())

# each patient folder is DATA_DIR/train/<PatientID>/
# we’ll just use the "FVC" from train.csv as labels
patient_dirs = {p: os.path.join(DATA_DIR, "train", p) for p in os.listdir(os.path.join(DATA_DIR, "train"))}

# merge csv + directory path
df["dicom_dir"] = df["Patient"].map(patient_dirs)
# keep only existing folders
df = df[df["dicom_dir"].apply(os.path.isdir)].reset_index(drop=True)

# make sure we only keep one FVC per patient (baseline)
df = df.groupby("Patient").first().reset_index()

print(f"Total patients found: {len(df)}")


                     Patient  Weeks   FVC    Percent  Age   Sex SmokingStatus
0  ID00007637202177411956430     -4  2315  58.253649   79  Male     Ex-smoker
1  ID00007637202177411956430      5  2214  55.712129   79  Male     Ex-smoker
2  ID00007637202177411956430      7  2061  51.862104   79  Male     Ex-smoker
3  ID00007637202177411956430      9  2144  53.950679   79  Male     Ex-smoker
4  ID00007637202177411956430     11  2069  52.063412   79  Male     Ex-smoker
Total patients found: 176


In [82]:
train_df = df.sample(frac=0.8, random_state=42)  # 80% train
val_df   = df.drop(train_df.index)               # 20% val

train_loader = DataLoader(OSICDataset(train_df), batch_size=2, shuffle=True)
val_loader   = DataLoader(OSICDataset(val_df),   batch_size=2, shuffle=False)
print(f"Train patients: {len(train_df)}, Val patients: {len(val_df)}")

Train patients: 141, Val patients: 35


In [92]:
model = OSICProbCNN()          # your CNN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
opt =  torch.optim.Adam(model.parameters(), lr=1e-3)

# loss_fn = GaussianNLL()

# for epoch in range(1, 6):      # e.g. train 5 epochs
#     model.train()
#     total_loss = 0
#     for x, y in train_loader:
#         opt.zero_grad()
#         mu, log_var = model(x)
#         loss = loss_fn(mu, log_var, y)
#         loss.backward()
#         opt.step()
#         total_loss += loss.item() * x.size(0)

#     avg_train = total_loss / len(train_loader.dataset)

#     # simple validation (no device, no AMP)
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for x, y in val_loader:
#             mu, log_var = model(x)
#             val_loss += loss_fn(mu, log_var, y).item() * x.size(0)
#     avg_val = val_loss / len(val_loader.dataset)

#     print(f"Epoch {epoch}: Train {avg_train:.4f} | Val {avg_val:.4f}")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/bk/6nc34r7d5d32rbmm3kk61g6w0000gn/T/ipykernel_3745/2436056553.py", line 4, in <module>
    opt =  torch.optim.Adam(model.parameters(), lr=1e-3)
  File "/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/torch/optim/adam.py", line 99, in __init__
    fused = group.setdefault("fused", None)
  File "/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/torch/optim/optimizer.py", line 377, in __init__
  File "/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/torch/_compile.py", line 27, in inner
    import torch._dynamo
  File "/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/torch/_dynamo/__init__.py", line 3, in <module>
    from . import convert_frame, eval_frame, resume_execution
  File "/opt/miniconda3/envs/cs3244/lib/python3.9/site-pack

### Data Exploration

In [ ]:
dir = "/Users/jinkyungjeon/cs3244_project/osic-pulmonary-fibrosis-progression"
train_csv = pd.read_csv(os.path.join(dir, 'train.csv'))
train_path = os.path.join(dir, 'train')

train_path = os.path.join(dir, 'train')
test_path = os.path.join(dir, 'test')

train_patients = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]
test_patients = [d for d in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, d))]

print(len(train_patients), len(test_patients))


In [ ]:
# Number of patients in the training tabular file 
train_csv = pd.read_csv(os.path.join(dir, 'train.csv'))
print(len(train_csv)) # prints the number of rows in the CSV file

# Number of patients in the training ct scan folder
train_patients = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]
print(len(train_patients)) # prints the number of patient folders in the training folder

### Simple CNN Pipeline 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_cnn(input_shape=(224,224,1)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='linear')(x)
    return models.Model(inputs, outputs)


/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.4 when it was built against 1.14.3, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


### Training CNN

In [ ]:
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import pandas as pd


X_train = load_dicom_images_from_directory(os.path.join(train_path, patient_id))
y_train = train_csv[train_csv['Patient'] == patient_id]['FVC'].values

X_test = load_dicom_images_from_directory(os.path.join(test_path, patient_id))
y_test = train_csv[train_csv['Patient'] == patient_id]['FVC'].values

# Build CNN
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='linear')  # regression for FVC
])

model.compile(optimizer='adam', loss='mae', metrics=['mse'])

# Suppose X_train and y_train are ready
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)


176 5


/opt/miniconda3/envs/cs3244/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
